In [1]:
import sys
import os
sys.path.insert(0, '../verificator')

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [47]:
video_path = '/home/shareduser/d_test/data/videos/cam_2'
img_saving_path = '/home/shareduser/d_test/data/pics/deep_cam_2_thr_0.5'
emb_saving_path ='/home/shareduser/d_test/data/models/embeddings'
numpy_saving_path = os.path.join(img_saving_path, 'deep_sort_features')

In [4]:
build_path = '/home/shareduser/d_test/data/models/yolo_init_config'
weights = os.path.join(build_path, 'bin', 'yolo.weights')
model = os.path.join(build_path, 'cfg', 'yolo-people.cfg')
cfg = os.path.join(build_path, 'cfg')
labels = os.path.join(build_path, 'labels.txt')
trained_model_path = '/home/shareduser/d_test/data/models/darkflow_retrain_all_imgs/full_data'

In [5]:
import numpy as np

In [6]:
from verificator.verificator import Verificator

In [7]:
from verificator.feature_extractor import DeepSortExtractor

In [8]:
deep_sort_model_path = '../verificator/verificator/deep_sort/resources/networks/mars-small128.ckpt-68577'

In [9]:
import tensorflow as tf

In [10]:
encoder = DeepSortExtractor(deep_sort_model_path)

feature dimensionality:  128


## Проверка детектора на видео cam 2

In [11]:
from verificator.deep_sort.deep_sort import nn_matching
from verificator.deep_sort.deep_sort.tracker import Tracker

In [58]:
metrics = nn_matching.NearestNeighborDistanceMetric('cosine', 0.1)
tracker = Tracker(metrics, max_age=10)

In [13]:
from verificator.detector_deep_sort import DeepDetector

In [14]:
# опции детектора
options=dict()
options['model'] = model
options['load'] = -1
options['config'] = cfg
options['labels'] = labels
options['gpu'] = 0.8
options['threshold'] = 0.5
options['backup'] = os.path.join(trained_model_path, 'ckpt')
options['summary'] = os.path.join(trained_model_path, 'summary')

In [15]:
from verificator.detector import Detector

In [16]:
detector = DeepDetector(options)

Parsing /home/shareduser/d_test/data/models/yolo_init_config/cfg/yolo-people.cfg
Loading None ...
Finished in 0.0002903938293457031s

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Init  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 76, 76, 256)
 Init  

In [59]:
if not os.path.exists(img_saving_path):
    os.mkdir(img_saving_path)

In [60]:
detector.detect_and_save(
    tracker,
    encoder,
    save_mode='images',
    video_file_path=video_path,
    img_saving_path=img_saving_path,
    show_video=False)

processing file: 170814_172001.avi
processing file: 170814_171801.avi
processing file: 170814_170101.avi
processing file: 170814_172501.avi
processing file: 170814_172801.avi
processing file: 170814_173701.avi
processing file: 170814_173001.avi
processing file: 170814_171501.avi
processing file: 170814_170701.avi
processing file: 170814_170201.avi
processing file: 170814_174001.avi
processing file: 170814_170801.avi
processing file: 170814_171001.avi
processing file: 170814_170301.avi
processing file: 170814_170901.avi
processing file: 170814_171701.avi
processing file: 170814_174601.avi
processing file: 170814_171101.avi
processing file: 170814_172701.avi
processing file: 170814_173801.avi
processing file: 170814_170501.avi
processing file: 170814_171401.avi
processing file: 170814_171201.avi
processing file: 170814_170601.avi
processing file: 170814_172101.avi
processing file: 170814_172201.avi
processing file: 170814_171301.avi
processing file: 170814_171601.avi
processing file: 170

### --проверили, работает---

## Проверка обучения tpe-верификатора

In [22]:
from verificator.feature_extractor import *

In [23]:
import tensorflow as tf

In [25]:
tf.reset_default_graph()

In [26]:
fextractor = DeepSortExtractor(deep_sort_model_path)

feature dimensionality:  128


In [27]:
from verificator.utils.clean_after_detections import clean_dirs

In [28]:
#clean_dirs(img_saving_path, clean_mode='with_aspect')

In [29]:
import numpy as np
import cv2

In [30]:
labels, features = fextractor.encode_from_dirs(img_saving_path)

In [31]:
features.shape

(6432, 128)

In [46]:

if not os.path.exists(numpy_saving_path):
    os.mkdir(numpy_saving_path)

In [47]:
# сохранили фичи, созданные в deep sort 
np.save(os.path.join(numpy_saving_path, 'img_features.npy'), features)
np.save(os.path.join(numpy_saving_path, 'labels.npy'), labels)

In [13]:
pred_to_train = TPEPredictor(128,128)

../verificator/verificator/tpe_model/tpe_model_builder.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, weights=[array([[ ..., activation="linear", input_dim=128, use_bias=False)`
  base_model.add(Dense(n_out, input_dim=n_in, bias=False, weights=[W_pca], activation='linear'))
../verificator/verificator/tpe_model/tpe_model_builder.py:49: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  e = merge([a_emb, p_emb, n_emb], mode=triplet_merge, output_shape=triplet_merge_shape)
/home/shareduser/.local/lib/python3.5/site-packages/keras/legacy/layers.py:460: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
../verificator/verificator/tpe_model/tpe_model_builder.py:50: UserWarning: Update your `Model` call to the Keras 2 API: `Model(input

In [14]:
tpe_weights_saving_path = '/home/shareduser/d_test/data/models/tpe_weights'

In [15]:
features = np.load(os.path.join(numpy_saving_path, 'img_features.npy'))
labels = np.load(os.path.join(numpy_saving_path, 'labels.npy'))

In [16]:
features.shape

(6432, 128)

In [17]:
mineer, mindeer = pred_to_train.train(
    10, 32, weights_saving_path=tpe_weights_saving_path, weights_saving_name='from_deep_sot_10_ep.h5',
    labels=labels, embeddings=features)

training tpe, number of train data (5209, 128), number of dev data (1160, 128), epochs 10
epoch: 0
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7208
min score: -0.2300722301006317, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6594
min score: -0.271414577960968, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6496
min score: -0.29595747590065, max score: 1.0000004768371582
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6535
min score: -0.319982647895813, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6708
min score: -0.32290586829185486, max score: 1.0000004768371582
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6810
min score: -0.33300212025642395, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6637
min score: -0.3409915864467621, max score: 1.000

Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6765
min score: -0.5587077736854553, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6685
min score: -0.562656581401825, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6490
min score: -0.5778520703315735, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7103
min score: -0.5513442754745483, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7029
min score: -0.5896587371826172, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6276
min score: -0.5841923952102661, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7362
min score: -0.5760201811790466, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6541
min score: -0.6

Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7014
min score: -0.6006110310554504, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6447
min score: -0.602379560470581, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6720
min score: -0.5646474361419678, max score: 1.0000004768371582
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6297
min score: -0.597167432308197, max score: 1.0000004768371582
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6622
min score: -0.6037406325340271, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6459
min score: -0.6055777668952942, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6733
min score: -0.6691105961799622, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7106
min score: -0.67

Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6455
min score: -0.6001409292221069, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6880
min score: -0.6001878976821899, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.7086
min score: -0.6178245544433594, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6655
min score: -0.594571053981781, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6523
min score: -0.6157028675079346, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6332
min score: -0.6119986772537231, max score: 1.0000004768371582
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6669
min score: -0.6445798873901367, max score: 1.0000004768371582
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6781
min score: -0.6

Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6449
min score: -0.6451536417007446, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6756
min score: -0.6346977353096008, max score: 1.0000004768371582
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6462
min score: -0.5392793416976929, max score: 1.0000004768371582
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6307
min score: -0.576511800289154, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6637
min score: -0.5862309336662292, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6281
min score: -0.5789585113525391, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6521
min score: -0.5724420547485352, max score: 1.0000008344650269
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6123
min score: -0.5

...

Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6615
min score: -0.6349595189094543, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6104
min score: -0.6093104481697083, max score: 1.0000004768371582
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6152
min score: -0.6111871004104614, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6140
min score: -0.5910264849662781, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6519
min score: -0.6037305593490601, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6332
min score: -0.5671424269676208, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6747
min score: -0.5602133870124817, max score: 1.0000005960464478
EER: 9.61
epoch: 7
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.

Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6501
min score: -0.5848457217216492, max score: 1.0000004768371582
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.5981
min score: -0.5693715214729309, max score: 1.0000004768371582
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6491
min score: -0.5768495798110962, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6427
min score: -0.5693286061286926, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6474
min score: -0.566527247428894, max score: 1.0000004768371582
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.5967
min score: -0.6319175958633423, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6555
min score: -0.6008455753326416, max score: 1.0000008344650269
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6281
min score: -0.5

Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6186
min score: -0.6282166838645935, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6365
min score: -0.6147194504737854, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6088
min score: -0.6087654829025269, max score: 1.0000004768371582
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6605
min score: -0.6373346447944641, max score: 1.0000004768371582
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6647
min score: -0.6250336170196533, max score: 1.0000004768371582
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6609
min score: -0.6174002289772034, max score: 1.0000004768371582
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6288
min score: -0.5574683547019958, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6324
min score: -0.

Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6066
min score: -0.5874102711677551, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6482
min score: -0.5836235284805298, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6576
min score: -0.5903476476669312, max score: 1.0000004768371582
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6504
min score: -0.5658639669418335, max score: 1.0000004768371582
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6112
min score: -0.5913010239601135, max score: 1.0000004768371582
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6296
min score: -0.5663674473762512, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6361
min score: -0.5853979587554932, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6160
min score: -0.

Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6379
min score: -0.5890043377876282, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6605
min score: -0.5675252676010132, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6185
min score: -0.577470064163208, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6260
min score: -0.5522611141204834, max score: 1.0000004768371582
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6873
min score: -0.5982242822647095, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6657
min score: -0.580281674861908, max score: 1.0000007152557373
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6377
min score: -0.5643667578697205, max score: 1.0000005960464478
Epoch 1/1
32/32 [==============================] - 0s - loss: 0.6467
min score: -0.57

In [37]:
f0=os.path.join(img_saving_path,os.listdir(img_saving_path)[0])
f1=os.path.join(img_saving_path,os.listdir(img_saving_path)[1])
f0, f1

('/home/shareduser/d_test/data/pics/cam_2_thr_0.5/1181',
 '/home/shareduser/d_test/data/pics/cam_2_thr_0.5/563')

In [41]:
img1 = os.path.join(f0, os.listdir(f0)[0])
img2 = os.path.join(f1, os.listdir(f1)[0])
distance = 0.24209733262206568
img1,img2

('/home/shareduser/d_test/data/pics/cam_2_thr_0.5/1181/1181_0.jpg',
 '/home/shareduser/d_test/data/pics/cam_2_thr_0.5/563/563_1.jpg')

In [31]:
pred_to_train.load_weights(os.path.join(tpe_weights_saving_path, 'from_deep_sot_10_ep.h5'))

In [32]:
v = Verificator(encoder, pred_to_train)

In [44]:
v.verify(img1,img2,distance)

(array([[False]], dtype=bool), array([[-0.024749]], dtype=float32))

In [18]:
pred_1 = TPEPredictor(128,128)
pred_2 = TPEPredictor(128,128)

../verificator/verificator/tpe_model/tpe_model_builder.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, input_dim=128, use_bias=False, weights=[array([[ ..., activation="linear")`
  base_model.add(Dense(n_out, input_dim=n_in, bias=False, weights=[W_pca], activation='linear'))
../verificator/verificator/tpe_model/tpe_model_builder.py:49: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  e = merge([a_emb, p_emb, n_emb], mode=triplet_merge, output_shape=triplet_merge_shape)
/home/shareduser/.local/lib/python3.5/site-packages/keras/legacy/layers.py:460: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
../verificator/verificator/tpe_model/tpe_model_builder.py:50: UserWarning: Update your `Model` call to the Keras 2 API: `Model(input

In [37]:
mineer, mindeer = predictor.train(1,32,sampling_batch_size=100,save_weights=False,labels=labels,embeddings=features)

epoch: 0


ValueError: Tensor("Variable:0", shape=(128, 128), dtype=float32_ref) must be from the same graph as Tensor("rho_1/read:0", shape=(), dtype=float32).

In [41]:
pred_2.load_weights(os.path.join(model_saving_path, model_saving_name))

In [39]:
v2 = Verificator(encoder, pred_1)

In [40]:
v2.verify(img1,img2,0.2678730307203351)

(array([[ True]], dtype=bool), array([[ 0.59078968]], dtype=float32))

In [42]:
v1 = Verificator(encoder, pred_2)

In [43]:
v1.verify(img1,img2,distance)

(array([[False]], dtype=bool), array([[ 0.12595434]], dtype=float32))

In [52]:
im1_enc = pred_2.predict(encoder.encode(cv2.imread(img1)))

In [53]:
im2_enc = pred_2.predict(encoder.encode(cv2.imread(img2)))

In [50]:
from sklearn.metrics.pairwise import cosine_similarity

In [54]:
cosine_similarity(im1_enc, im2_enc)

array([[ 0.12595436]], dtype=float32)